In [ ]:
from pathlib import Path
from scipy.io import wavfile
import scipy.signal
import pandas as pd
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

## Mount Drive

In [ ]:
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ON_COLAB = is_running_on_colab()
ON_COLAB

True

In [ ]:
if ON_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  intermediate_folder = Path('/content/gdrive/MyDrive/Temp/Speech recognition project')
else:
  intermediate_folder = Path('..') / 'data' / 'intermediate'


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Read files

In [ ]:
X_train = np.load(intermediate_folder / 'train_main_1_sec_audio.npy')
X_train.shape

(33566, 16000)

In [ ]:
X_val = np.load(intermediate_folder / 'val_main_1_sec_audio.npy')
X_val.shape

(4619, 16000)

In [ ]:
# X_test = np.load(intermediate_folder / 'test_main_1_sec_audio.npy')
# X_test.shape

In [ ]:
y_train_labels = pd.read_csv(intermediate_folder / 'train_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_val_labels = pd.read_csv(intermediate_folder / 'val_main_1_sec_labels.csv', header=None, index_col=False)[0]
# y_test_labels = pd.read_csv(intermediate_folder / 'test_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_train_labels.shape, y_val_labels.shape
# y_train_labels.shape, y_val_labels.shape, y_test_labels.shape

((33566,), (4619,))

In [ ]:
le = LabelEncoder()
le.fit(y_train_labels)

y_train = le.transform(y_train_labels)
y_val = le.transform(y_val_labels)
# y_test = le.transform(y_test_labels)
y_train.shape, y_val.shape
# y_train.shape, y_val.shape, y_test.shape

((33566,), (4619,))

In [ ]:
pd.Series(y_train).value_counts().sort_index()

0     1667
1     1655
2     1696
3     1662
4     1647
5     1683
6     1723
7     1630
8     1668
9     1650
10    1672
11    1687
12    1708
13    1727
14    1715
15    1672
16    1693
17    1591
18    1686
19    1734
dtype: int64

## RNN - 50

In [ ]:
lstm = Sequential([LSTM(50, input_shape=(16000, 1)),
                  Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 20)                1020      
                                                                 
Total params: 11,420
Trainable params: 11,420
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 16000, 1), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:, :, None], y_train, batch_size=128, epochs=20, validation_data=(X_val[:, :, None], y_val), callbacks=[early_stopping]);

Epoch 1/20
263/263 [==============================] - 132s 496ms/step - loss: 2.9927 - accuracy: 0.0577 - val_loss: 2.9943 - val_accuracy: 0.0595
Epoch 2/20
263/263 [==============================] - 158s 601ms/step - loss: 2.9896 - accuracy: 0.0575 - val_loss: 2.9911 - val_accuracy: 0.0520
Epoch 3/20
263/263 [==============================] - 133s 507ms/step - loss: 2.9866 - accuracy: 0.0616 - val_loss: 2.9897 - val_accuracy: 0.0569
Epoch 4/20
263/263 [==============================] - 130s 495ms/step - loss: 2.9854 - accuracy: 0.0606 - val_loss: 2.9896 - val_accuracy: 0.0541
Epoch 5/20
263/263 [==============================] - 130s 492ms/step - loss: 2.9816 - accuracy: 0.0656 - val_loss: 2.9880 - val_accuracy: 0.0567
Epoch 6/20
263/263 [==============================] - 129s 492ms/step - loss: 2.9859 - accuracy: 0.0644 - val_loss: 2.9898 - val_accuracy: 0.0602
Epoch 7/20
263/263 [==============================] - 130s 492ms/step - loss: 2.9830 - accuracy: 0.0643 - val_loss: 2.9898 -

In [ ]:
# learning_rate = 0.005
lstm.fit(X_train[:, :, None], y_train, batch_size=128, epochs=10, validation_data=(X_val[:, :, None], y_val), callbacks=[early_stopping]);

Epoch 1/10
263/263 [==============================] - 132s 494ms/step - loss: 2.9932 - accuracy: 0.0628 - val_loss: 2.9923 - val_accuracy: 0.0645
Epoch 2/10
263/263 [==============================] - 132s 501ms/step - loss: 2.9894 - accuracy: 0.0631 - val_loss: 2.9987 - val_accuracy: 0.0647
Epoch 3/10
263/263 [==============================] - 127s 483ms/step - loss: 2.9867 - accuracy: 0.0656 - val_loss: 2.9920 - val_accuracy: 0.0589
Epoch 4/10
263/263 [==============================] - 130s 495ms/step - loss: 2.9846 - accuracy: 0.0641 - val_loss: 2.9919 - val_accuracy: 0.0585
Epoch 5/10
263/263 [==============================] - 129s 491ms/step - loss: 2.9832 - accuracy: 0.0638 - val_loss: 2.9937 - val_accuracy: 0.0554
Epoch 6/10
263/263 [==============================] - 130s 492ms/step - loss: 2.9837 - accuracy: 0.0649 - val_loss: 2.9933 - val_accuracy: 0.0541
Epoch 7/10
263/263 [==============================] - 130s 493ms/step - loss: 2.9841 - accuracy: 0.0649 - val_loss: 2.9984 -

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:, :, None], y_train, batch_size=128, epochs=20, validation_data=(X_val[:, :, None], y_val), callbacks=[early_stopping]);

Epoch 1/20
263/263 [==============================] - 190s 713ms/step - loss: 3.0094 - accuracy: 0.0518 - val_loss: 3.0038 - val_accuracy: 0.0517
Epoch 2/20
263/263 [==============================] - 185s 703ms/step - loss: 2.9967 - accuracy: 0.0561 - val_loss: 2.9956 - val_accuracy: 0.0541
Epoch 3/20
263/263 [==============================] - 185s 704ms/step - loss: 2.9909 - accuracy: 0.0597 - val_loss: 2.9963 - val_accuracy: 0.0608
Epoch 4/20
263/263 [==============================] - 186s 707ms/step - loss: 2.9876 - accuracy: 0.0611 - val_loss: 2.9918 - val_accuracy: 0.0602
Epoch 5/20
263/263 [==============================] - 185s 704ms/step - loss: 2.9857 - accuracy: 0.0631 - val_loss: 2.9878 - val_accuracy: 0.0552
Epoch 6/20
263/263 [==============================] - 185s 705ms/step - loss: 2.9834 - accuracy: 0.0620 - val_loss: 2.9892 - val_accuracy: 0.0595
Epoch 7/20
263/263 [==============================] - 193s 734ms/step - loss: 2.9799 - accuracy: 0.0658 - val_loss: 2.9902 -

## RNN - 100 + Fully connected 100 + Bidirectional

In [ ]:
lstm = Sequential([Bidirectional(LSTM(70), input_shape=(16000, 1)),
                   Dense(100, activation='sigmoid'),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 140)              40320     
 l)                                                              
                                                                 
 dense (Dense)               (None, 100)               14100     
                                                                 
 dense_1 (Dense)             (None, 20)                2020      
                                                                 
Total params: 56,440
Trainable params: 56,440
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 16000, 1), (None, 20))

In [ ]:
callbacks = [EarlyStopping(monitor='loss', patience=4, min_delta=.001),
             EarlyStopping(monitor='val_loss', patience=10, min_delta=.001)]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:, :, None], y_train, batch_size=128, epochs=20, validation_data=(X_val[:, :, None], y_val), callbacks=[early_stopping]);

Epoch 1/20
263/263 [==============================] - 324s 1s/step - loss: 3.0073 - accuracy: 0.0536 - val_loss: 3.0020 - val_accuracy: 0.0539
Epoch 2/20
263/263 [==============================] - 323s 1s/step - loss: 2.9946 - accuracy: 0.0575 - val_loss: 2.9982 - val_accuracy: 0.0539
Epoch 3/20
263/263 [==============================] - 326s 1s/step - loss: 2.9918 - accuracy: 0.0601 - val_loss: 2.9940 - val_accuracy: 0.0533
Epoch 4/20
263/263 [==============================] - 322s 1s/step - loss: 2.9855 - accuracy: 0.0633 - val_loss: 2.9871 - val_accuracy: 0.0591
Epoch 5/20
263/263 [==============================] - 331s 1s/step - loss: 2.9831 - accuracy: 0.0660 - val_loss: 2.9965 - val_accuracy: 0.0617
Epoch 6/20
263/263 [==============================] - 324s 1s/step - loss: 2.9847 - accuracy: 0.0669 - val_loss: 2.9949 - val_accuracy: 0.0630
Epoch 7/20
263/263 [==============================] - 323s 1s/step - loss: 2.9845 - accuracy: 0.0660 - val_loss: 2.9957 - val_accuracy: 0.0572

KeyboardInterrupt: ignored

## RNN - 2 bidirectional LSTM + Fully connected 100

In [ ]:
lstm = Sequential([Bidirectional(LSTM(32, return_sequences=True), input_shape=(16000, 1)),
                  #  Dropout(.5),
                   Bidirectional(LSTM(32)),
                  #  Dropout(.5),
                   Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 16000, 64)        8704      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 100)               6500      
                                                                 
 dense_1 (Dense)             (None, 20)                2020      
                                                                 
Total params: 42,056
Trainable params: 42,056
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 16000, 1), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=10, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:, :, None], y_train, batch_size=64, epochs=20, validation_data=(X_val[:, :, None], y_val),
         callbacks=[early_stopping]);

Epoch 1/20
 23/525 [>.............................] - ETA: 12:55:24 - loss: 3.0537 - accuracy: 0.0510

KeyboardInterrupt: ignored

## Report

In [ ]:
y_pred_lstm = lstm.predict(X_val[:, :, None])
print(classification_report(y_val, y_pred_lstm.argmax(axis=1)))

145/145 [==============================] - 29s 197ms/step
              precision    recall  f1-score   support

           0       0.01      0.00      0.01       247
           1       0.10      0.03      0.04       226
           2       0.07      0.05      0.06       224
           3       0.00      0.00      0.00       261
           4       0.08      0.05      0.06       231
           5       0.05      0.03      0.03       233
           6       0.00      0.00      0.00       215
           7       0.07      0.01      0.02       235
           8       0.00      0.00      0.00       232
           9       0.04      0.09      0.06       234
          10       0.00      0.00      0.00       209
          11       0.03      0.03      0.03       235
          12       0.06      0.12      0.08       240
          13       0.07      0.16      0.10       241
          14       0.02      0.00      0.01       225
          15       0.05      0.01      0.02       225
          16       0.05